# Train MNIST 

MNIST is the "Hello World" of image classification.
This notebook is almost a carbon copy of [this](https://colab.research.google.com/github/rpi-techfundamentals/fall2018-materials/blob/master/10-deep-learning/04-pytorch-mnist.ipynb#scrollTo=p_K4RdAT4nGA) notebook by Analytics Dojo with some intentional errors built in.

In [1]:
#Import Libraries


import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable



In [2]:

DATA_DIR='../data'
TRAIN_BATCH=256
TEST_BATCH=1000
EPOCHS=2
LEARNING_RATE=0.01
MOMENTUM=0.5
RANDOM_SEED=42
CUDA=False
IMAGE_NORM=(0.1307,), (0.3081,)


In [3]:
#load the data
def get_data_loader(is_train, batch_size, download=False):
    loader = torch.utils.data.DataLoader(
        datasets.MNIST(DATA_DIR, train=is_train, download=download,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(*IMAGE_NORM)
                       ])),
        batch_size=batch_size, shuffle=True)
    return loader
    

train_loader = get_data_loader(True, TRAIN_BATCH, download=True)
test_loader = get_data_loader(False, TEST_BATCH)


In [4]:


class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        NUM_CLASSES = 10 #the num of classes should be 10
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, NUM_CLASSES)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)


In [7]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if CUDA:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if CUDA:
            data, target = data.cuda(), target.cuda()
        #UserWarning: volatile was removed and now has no effect. Use with torch.no_grad(): instead.
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').data # sum up batch loss
        #size_average and reduce args will be deprecated, please use reduction='sum' instead.
        
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        # Comparing the predictions with targets, counting correct predictions, and adding them to total.
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




In [ ]:
#NUM_CLASSES = len(torch.unique(target)) , the number of classes should be 10
# for batch_idx, (data, target) in enumerate(train_loader):
#     target = Variable(target)
#     print(len(torch.unique(target)))
#     break

In [ ]:
model = Net()
if CUDA:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.335613
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.206762
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.924677
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.326306
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.944735

Test set: Average loss: 0.4984, Accuracy: 8758/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.855104
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.831246
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.686011
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.717873
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.579451
